<b>Importing Wikipedia page and transform it in Dataframe</b>

In [2]:
import pandas as pd
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',index_col=1)
df = tables[0]

<b>Renaming Collumns names</b>

In [3]:
df1=df.reset_index().rename(columns={"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"})

<b>Removing rows where Borough cell is Not Assigned</b>

In [4]:
df2=df1[df1.Borough != "Not assigned"]

<b>Grouping neighborhoods with the same Borough and PostalCode</b>

In [28]:
df3=df2.groupby(['Borough','PostalCode'])['Neighborhood'].apply(','.join).reset_index()
df3.head()

,Borough,PostalCode,Neighborhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,North Toronto West
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park,Summerhill East"


<b>Replacing cells where Neighborhood value is set Not Assigned to Neighborhoods name</b>

In [30]:
import numpy as np
df3['Neighborhood'] = np.where(df3['Neighborhood'] == 'Not assigned', df3['Borough'], df3['Neighborhood'])
df3

,Borough,PostalCode,Neighborhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,North Toronto West
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park,Summerhill East"
5,Central Toronto,M4V,"Deer Park,Forest Hill SE,Rathnelly,South Hill,..."
6,Central Toronto,M5N,Roselawn
7,Central Toronto,M5P,"Forest Hill North,Forest Hill West"
8,Central Toronto,M5R,"The Annex,North Midtown,Yorkville"
9,Downtown Toronto,M4W,Rosedale


<b>Checking number of rows in dataframe</b>

In [33]:
df3.shape

(103, 3)

<b>Merging Postal Code Collumns from both dataframes</b>

In [62]:
ref = pd.read_csv('https://cocl.us/Geospatial_data')
ref.rename(columns={"Postal Code": "PostalCode"},inplace=True)
df_final = pd.merge(df3, ref, on='PostalCode')
df_final = df_final[df_final['Borough'].str.contains("Toronto")]

<b>Setting FourSquare search</b>

In [50]:
CLIENT_ID = '5B3ANSER2PE4VVED01SWWJ1BE2DDFO0FI0V5SLHC5M51WKTR' # your Foursquare ID
CLIENT_SECRET = '3NTOOS53ZAFDOV1LGGURRPKH5UE0FGU2I3EO1LTP0LRQB4NL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [51]:
import requests
results = requests.get(url).json()

In [52]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [58]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
toronto_venues = getNearbyVenues(names=df_final['Neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

In [65]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
#    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
#    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#    print('\n')

In [66]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [89]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [90]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Borough,PostalCode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790,1,Park,Bus Line,Swim School,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197,4,Park,Gym,Grocery Store,Breakfast Spot,Clothing Store,Food & Drink Shop,Sandwich Place,Hotel,Doner Restaurant,Dim Sum Restaurant
2,Central Toronto,M4R,North Toronto West,43.715383,-79.405678,4,Coffee Shop,Yoga Studio,Bagel Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Dessert Shop,Park,Clothing Store,Chinese Restaurant
3,Central Toronto,M4S,Davisville,43.704324,-79.388790,4,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Restaurant,Café,Italian Restaurant,Sushi Restaurant,Thai Restaurant,Park
4,Central Toronto,M4T,"Moore Park,Summerhill East",43.689574,-79.383160,2,Tennis Court,Playground,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [94]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library


# create map
map_clusters = folium.Map(location=[43.65, -79.38], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<b>From above map one can observe that in general neighborhoods that are a similar distance from the center of downtown toronto are are clustered together. Since the criteria for clustering was similarity in venues for the neighborhoods this implies that similar venues are within neighborhoods of similar distance from center of Toronto. This then also might imply that certain types of customers either live or frequent venues in those locations creating the demand for the same type of venues in those neighborhoods.</b>